In [1]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Text classification

## Load Tweets dataset

Start by loading the Tweets dataset from the 🤗 Datasets library:

In [2]:
from datasets import load_dataset

tweets = load_dataset("tweet_eval", "emotion")

In [10]:
# Returns train split of data set as json
def _get_data(data):
    _train_data = data["train"]
    
    _all_data = []
    for row in _train_data:
        _all_data.append(row)
        
    return _all_data

In [16]:
def _data_type(data):
    _features = data["train"].features
    
    _types = {}
    for f in _features:
        _ftype = _features[f].dtype
        
        if _ftype == "int64":
            _fnames = _features[f].names
            
            _str2int = {}
            _int2str = {}
            _ints = []
            for n in _fnames:
                _nint = _features[f].str2int(n)
                _str2int[n] = _nint
                _int2str[_nint] = n
                _ints.append(_nint)
            
            _types[f] = {"type":_ftype, "names":_fnames, "ints":_ints, "int2str":_int2str, "str2int":_str2int}
            
        else:
            _types[f] = {"type":_ftype}
    return _types

_data_type(tweets)

{'text': {'type': 'string'},
 'label': {'type': 'int64',
  'names': ['anger', 'joy', 'optimism', 'sadness'],
  'ints': [0, 1, 2, 3],
  'int2str': {0: 'anger', 1: 'joy', 2: 'optimism', 3: 'sadness'},
  'str2int': {'anger': 0, 'joy': 1, 'optimism': 2, 'sadness': 3}}}

Then take a look at an example:

In [6]:
tweets["train"][0]

{'text': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'label': 2}

There are two fields in this dataset:

- `text`: the tweet text.
- `label`: a value where 0: anger, 1: joy, 2: optimism, 3: sadness



## Preprocess

The next step is to load a DistilBERT tokenizer to preprocess the `text` field:

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Create a preprocessing function to tokenize `text` and truncate sequences to be no longer than DistilBERT's maximum input length:

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) function. You can speed up `map` by setting `batched=True` to process multiple elements of the dataset at once:

In [6]:
tokenized_tweets = tweets.map(preprocess_function, batched=True)

Map:   0%|          | 0/1421 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [accuracy](https://huggingface.co/spaces/evaluate-metric/accuracy) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [8]:
!pip install evaluate

In [9]:
import evaluate

accuracy = evaluate.load("accuracy")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the accuracy:

In [10]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

Before you start training your model, create a map of the expected ids to their labels with `id2label` and `label2id`:

In [11]:
id2label = {0: "ANGER", 1: "JOY", 2: "OPTIMISM", 3: "SADNESS"}
label2id = {"ANGER": 0, "JOY": 1, "OPTIMISM": 2, "SADNESS": 3}

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load DistilBERT with [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSequenceClassification) along with the number of expected labels, and the label mappings:

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=4, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


At this point, only three steps remain:

1. Define your training hyperparameters in [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the accuracy and save the training checkpoint.
2. Pass the training arguments to [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [13]:
! pip install -U accelerate
! pip install -U transformers

In [15]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_tweets["train"],
    eval_dataset=tokenized_tweets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.640028,0.780436
2,No log,0.586313,0.796622


TrainOutput(global_step=408, training_loss=0.7361039554371553, metrics={'train_runtime': 1871.9101, 'train_samples_per_second': 3.48, 'train_steps_per_second': 0.218, 'total_flos': 70661199833448.0, 'train_loss': 0.7361039554371553, 'epoch': 2.0})

<Tip>

[Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) applies dynamic padding by default when you pass `tokenizer` to it. In this case, you don't need to specify a data collator explicitly.

</Tip>

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [18]:
# from huggingface_hub import notebook_login

# notebook_login()

In [28]:
# trainer.push_to_hub()
trainer.save_model()

<Tip>

For a more in-depth example of how to finetune a model for text classification, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb)
or [TensorFlow notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification-tf.ipynb).

</Tip>

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Grab some text you'd like to run inference on:

In [20]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for sentiment analysis with your model, and pass your text to it:

In [29]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="my_awesome_model")
classifier(text)

[{'label': 'JOY', 'score': 0.771780252456665}]